## Роевой интеллект: муравьи

В этой части я реализую муравьиный алгоритм — метод оптимизации, вдохновлённый поведением реальных муравьёв при поиске кратчайшего пути к источнику пищи. Алгоритм особенно хорошо подходит для задач коммивояжёра, поиска маршрутов и других комбинаторных задач.

В качестве практической задачи рассматривается планирование оптимального маршрута по парку развлечений Диснейленд, чтобы минимизировать общее время ожидания в очередях. Для этого я использую актуальные данные с сайта queue-times.com, который предоставляет информацию о текущем времени ожидания на аттракционы.

Каждый «муравей» в алгоритме будет моделировать посетителя парка, пытающегося найти лучший маршрут по аттракционам с учётом текущей загруженности. Муравьи будут оставлять виртуальные феромоны на маршрутах, которые приводят к более короткому времени ожидания, тем самым усиливая привлекательность удачных путей для последующих агентов.

Таким образом, цель алгоритма — найти наиболее выгодную последовательность посещения аттракционов, которая минимизирует время в очередях и делает пребывание в парке максимально эффективным и комфортным.

In [25]:
%use kandy
%use ktor-client

### Подготовка функций и данных о парке

In [26]:
@Serializable
data class Park(
  val id: Int,
  val name: String,
  val country: String,
  val continent: String,
  val latitude: Double,
  val longitude: Double,
  val timezone: String
)

@Serializable
data class Company(
  val id: Int,
  val name: String,
  val parks: List<Park>
)

In [27]:
fun fetchParksData(): List<Company> =
  Json.decodeFromString(http.get("https://queue-times.com/parks.json").body())

fetchParksData()
  .find { it.name.contains("Disney") }
  ?.parks
  ?.map { it.name }

[Animal Kingdom, Disney California Adventure, Disney Hollywood Studios, Disney Magic Kingdom, Disneyland, Disneyland Hong Kong, Disneyland Park Paris, Epcot, Shanghai Disney Resort, Tokyo Disneyland, Tokyo DisneySea, Walt Disney Studios Paris]

In [28]:
fetchParksData()
  .find { it.name.contains("Disney") }
  ?.parks
  ?.find { it.name.contains("Shanghai") }

Park(id=30, name=Shanghai Disney Resort, country=China, continent=Asia, latitude=31.144, longitude=121.657, timezone=Asia/Shanghai)

In [29]:
@Serializable
data class Ride(
  val id: Int,
  val name: String,
  val is_open: Boolean,
  val wait_time: Int,
  val last_updated: String,
)

@Serializable
data class Land(
  val id: Int,
  val name: String,
  val rides: List<Ride>
)

@Serializable
data class QueuesInfo(
  val lands: List<Land>,
  val rides: List<Ride>,
)

In [30]:
fun fetchParkQueueTimesData(parkId: Int): QueuesInfo =
  Json.decodeFromString(http.get("https://queue-times.com/parks/$parkId/queue_times.json").body())

fetchParksData()
  .find { it.name.contains("Disney") }
  ?.parks
  ?.find { it.name.contains("California") }
  ?.run { fetchParkQueueTimesData(parkId = id) }
  ?.lands
  ?.map { it.rides }
  ?.flatten()
  ?.map { "${it.name}: ожидание ${it.wait_time} мин."}

[Guardians of the Galaxy - Mission: BREAKOUT!: ожидание 30 мин., WEB SLINGERS: A Spider-Man Adventure: ожидание 35 мин., WEB SLINGERS: A Spider-Man Adventure Single Rider: ожидание 0 мин., Luigi's Rollickin' Roadsters: ожидание 15 мин., Mater's Junkyard Jamboree: ожидание 5 мин., Radiator Springs Racers: ожидание 60 мин., Radiator Springs Racers Single Rider: ожидание 0 мин., Grizzly River Run: ожидание 45 мин., Redwood Creek Challenge Trail: ожидание 0 мин., Soarin' Around the World: ожидание 60 мин., Animation Academy: ожидание 0 мин., Mickey's PhilharMagic: ожидание 10 мин., Monsters, Inc. Mike & Sulley to the Rescue!: ожидание 40 мин., Sorcerer's Workshop: ожидание 0 мин., Turtle Talk with Crush: ожидание 0 мин., Golden Zephyr: ожидание 0 мин., Goofy's Sky School: ожидание 0 мин., Silly Symphony Swings: ожидание 5 мин., Silly Symphony Swings Single Rider: ожидание 0 мин., The Little Mermaid - Ariel's Undersea Adventure: ожидание 15 мин., Games of Pixar Pier: ожидание 5 мин., Incred

In [31]:
data class ExtendedRide(
  val ride: Ride,
  val x: Double,
  val y: Double,
)

In [32]:
fun generateRides(
  companyName: String,
  parkName: String
): List<ExtendedRide> {
  val rides = fetchParksData()
    .find { it.name.contains(companyName) }
    ?.parks
    ?.find { it.name.contains(parkName) }
    ?.run { fetchParkQueueTimesData(parkId = id) }
    ?.lands
    ?.map { it.rides }
    ?.flatten()
    ?.filter { it.is_open }
    .orEmpty()

  val count = rides.count()

  return rides.mapIndexed { index, ride ->
    ExtendedRide(
      ride = ride,
      x = sin(index * 1.0 / count * PI * 2) * Math.random() * 2,
      y = cos(index * 1.0 / count * PI * 2) * Math.random() * 2,
    )
  }
}

In [33]:
val rides = generateRides(companyName = "Disney", parkName = "California")

plot {
  points {
    x(rides.map { it.x })
    y(rides.map { it.y })
  }

  layout {
    title = "Координаты аттракционов"
  }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="CHCxBu"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Координаты аттракционов"
},
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":[0.0,0.38635349382600037,0.7528694674299101,0.08463830733471223,0.6749376299146473,0.4916159026762682,1.5014635514561858,0.26244723270546233,0.39747743739178276,0.5761553972498336,0.7080028453141961,0.3605721729174844,0.2324369355967305,0.3398574402388374,8.343185238400054E-17,-0.40198098443549957,-0.7632364697713713,-0.711978892353423,-0.06472787273892143,-0.8289061613699463,-0.9469673120626054,-1.8613992653232292,-1.3239821327367545,-0.5101799845352926,-0.34343188775203426,-1.058159007837918,-0.32394100597600956,-0.32099848004323006],
"y":[1.9780234351173387,1.0136348036395537,0.8896870660588203,1.532234644735529,0.9037790156368312,0.780402850469572,0.3070726811781597,8.4989569053696E-17,-0.05535365388779097,-0.4358303232190818,-0.8888682907777968,-0.10626222145523498,-1.4079005476847177,-0.1853524175250885,-0.32023723237058954,-1.4999056275900493,-1.1977096637173936,-0.8233023380742879,-1.141764592320307,-0.4888561192659585,-0.010571571073118503,-1.0699019423890383E-17,0.0015975242667319591,0.2774068264758736,0.8066501777106571,0.6239174911965072,1.7784916032531084,1.202080881399731]
},
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"point",
"data_meta":{
"series_annotations":[{
"type":"float",
"column":"x"
},{
"type":"float",
"column":"y"
}]
}
}],
"spec_id":"8"
};
 var containerDiv = document.getElementById("CHCxBu");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 600.0,
 height: 400.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -2.0 
 
 
 
 
 
 
 
 
 -1.5 
 
 
 
 
 
 
 
 
 -1.0 
 
 
 
 
 
 
 
 
 -0.5 
 
 
 
 
 
 
 
 
 0.0 
 
 
 
 
 
 
 
 
 0.5 
 
 
 
 
 
 
 
 
 1.0 
 
 
 
 
 
 
 
 
 1.5 
 
 
 
 
 
 
 
 
 
 
 -1.5 
 
 
 
 
 
 
 -1.0 
 
 
 
 
 
 
 -0.5 
 
 
 
 
 
 
 0.0 
 
 
 
 
 
 
 0.5 
 
 
 
 
 
 
 1.0 
 
 
 
 
 
 
 1.5 
 
 
 
 
 
 
 2.0 
 
 
 
 
 
 
 
 
 Координаты аттракционов 
 
 
 
 
 y 
 
 
 
 
 x

In [34]:
fun getDistanceBetweenRides(
  rideA: ExtendedRide,
  rideB: ExtendedRide
) = Math.sqrt(rideA.x * rideB.x + rideA.y * rideB.y)

In [35]:
fun initializeMatrices(rides: List<ExtendedRide>): Pair<Array<DoubleArray>, Array<DoubleArray>> {
  val n = rides.size
  val distances = Array(n) { i ->
    DoubleArray(n) { j ->
      when {
        i == j -> Double.MAX_VALUE
        else -> getDistanceBetweenRides(rides[i], rides[j]).coerceAtLeast(0.1)  // Минимальное расстояние
      }
    }
  }
  val pheromones = Array(n) { DoubleArray(n) { 1.0 } }
  return distances to pheromones
}

fun greedyRoute(rides: List<ExtendedRide>, distances: Array<DoubleArray>): List<Int> {
  val route = mutableListOf(0)
  val unvisited = (1 until rides.size).toMutableSet()

  while (unvisited.isNotEmpty()) {
    val last = route.last()
    val next = unvisited.minByOrNull { distances[last][it] } ?: break
    route.add(next)
    unvisited.remove(next)
  }

  return route
}

fun buildAntRoute(
  n: Int,
  distances: Array<DoubleArray>,
  pheromones: Array<DoubleArray>,
  alpha: Double,
  beta: Double
): List<Int> {
  val route = mutableListOf((0 until n).random())
  val unvisited = (0 until n).toMutableSet().apply { remove(route.first()) }

  while (unvisited.isNotEmpty()) {
    val last = route.last()
    val next = selectNextNode(last, unvisited, distances, pheromones, alpha, beta)
    route.add(next)
    unvisited.remove(next)
  }

  return route
}

fun selectNextNode(
  from: Int,
  unvisited: Set<Int>,
  distances: Array<DoubleArray>,
  pheromones: Array<DoubleArray>,
  alpha: Double,
  beta: Double
): Int {
  // Комбинированная жадная+вероятностная стратегия
  val probabilities = unvisited.map { to ->
    val pheromone = pheromones[from][to].pow(alpha)
    val heuristic = (1.0 / distances[from][to]).pow(beta)
    pheromone * heuristic
  }

  // С вероятностью 30% выбираем самый лучший вариант
  if (Math.random() < 0.3) {
    return unvisited.elementAt(probabilities.indexOf(probabilities.maxOrNull()!!))
  }

  // Иначе вероятностный выбор
  val total = probabilities.sum()
  val rand = Math.random() * total
  var sum = 0.0

  probabilities.forEachIndexed { index, prob ->
    sum += prob
    if (rand <= sum) return unvisited.elementAt(index)
  }

  return unvisited.first()
}

fun calculateRouteCost(route: List<Int>, distances: Array<DoubleArray>): Double {
  return route.zipWithNext { a, b -> distances[a][b] }.sum()
}

fun updatePheromones(
  pheromones: Array<DoubleArray>,
  routes: List<List<Int>>,
  distances: Array<DoubleArray>,
  evaporationRate: Double,
  q: Double
) {
  // Испарение
  pheromones.forEach { row ->
    row.indices.forEach { col ->
      row[col] *= (1 - evaporationRate)
    }
  }

  // Только 2 лучших маршрута добавляют феромоны
  routes.sortedBy { calculateRouteCost(it, distances) }
    .take(2)
    .forEach { route ->
      val delta = q / calculateRouteCost(route, distances)
      route.zipWithNext { from, to ->
        pheromones[from][to] += delta
        pheromones[to][from] += delta
      }
    }
}

In [36]:
fun antColonyOptimizeRides(
  rides: List<ExtendedRide>,
  nAnts: Int = 10,
  nIterations: Int = 20,
  alpha: Double = 1.0,
  beta: Double = 2.0, // Увеличиваем важность расстояния
  evaporationRate: Double = 0.3,
  q: Double = 100.0
): List<ExtendedRide> {
  if (rides.size <= 1) return rides

  // 1. Быстрая инициализация матриц
  val (distances, pheromones) = initializeMatrices(rides)

  // 2. Начинаем с жадного маршрута как отправной точки
  var bestRoute = greedyRoute(rides, distances)
  var bestCost = calculateRouteCost(bestRoute, distances)

  repeat(nIterations) {
    val routes = (0 until nAnts).map {
      buildAntRoute(rides.size, distances, pheromones, alpha, beta)
    }

    // 3. Быстрое обновление феромонов
    updatePheromones(pheromones, routes, distances, evaporationRate, q)

    // 4. Проверяем только лучший маршрут в итерации
    routes.minByOrNull { calculateRouteCost(it, distances) }?.let { route ->
      val cost = calculateRouteCost(route, distances)
      if (cost < bestCost) {
        bestRoute = route
        bestCost = cost
      }
    }
  }

  return bestRoute.map { rides[it] }
}

In [37]:
val rides = generateRides(companyName = "Disney", parkName = "California")
val bestRoute = antColonyOptimizeRides(rides)

bestRoute.forEachIndexed { i, r ->
  println("${i + 1}) ${r.ride.name} — ожидание: ${r.ride.wait_time} мин.")
}

1) Guardians of the Galaxy - Mission: BREAKOUT! — ожидание: 30 мин.
2) Grizzly River Run — ожидание: 45 мин.
3) Sorcerer's Workshop — ожидание: 0 мин.
4) Incredicoaster Single Rider — ожидание: 0 мин.
5) Jumpin' Jellyfish — ожидание: 5 мин.
6) Games of Pixar Pier — ожидание: 5 мин.
7) Jessie's Critter Carousel — ожидание: 5 мин.
8) WEB SLINGERS: A Spider-Man Adventure — ожидание: 35 мин.
9) Toy Story Midway Mania! — ожидание: 40 мин.
10) The Little Mermaid - Ariel's Undersea Adventure — ожидание: 15 мин.
11) Animation Academy — ожидание: 0 мин.
12) Turtle Talk with Crush — ожидание: 0 мин.
13) Redwood Creek Challenge Trail — ожидание: 0 мин.
14) Radiator Springs Racers — ожидание: 60 мин.
15) The Bakery Tour — ожидание: 0 мин.
16) Mater's Junkyard Jamboree — ожидание: 5 мин.
17) Pixar Pal-A-Round – Non-Swinging — ожидание: 20 мин.
18) Inside Out Emotional Whirlwind — ожидание: 10 мин.
19) Pixar Pal-A-Round - Swinging — ожидание: 30 мин.
20) Luigi's Rollickin' Roadsters — ожидание: 15 м